## Overview
The goal for this repo is to build a multi-agent customer service system where specialized agents coordinate using Agent-to-Agent (A2A) communication and access customer data through the Model Context Protocol (MCP).  
HERE IS THE GITHUB REPO for this project:

https://github.com/LatifaTam/Multi-Agent-Customer-Service-System-with-A2A-and-MCP-Overview

### 0. Conclusion
Throughout the implementation of this multi-agent system, several interesting challenges came up. One of the biggest observations was how strict ADK is when handling tools. Any mismatch in required parameters or slight formatting issues immediately causes tool failures, producing noisy errors that can be hard to diagnose. It reinforced how important it is to give the LLM extremely explicit tool instructions so it does not make up parameters or misname anything.

The other recurring issue was the NoneType error when reading task artifacts. Sometimes ADK produces incomplete artifact structures during the first execution, which caused the "NoneType is not subscriptable" exception. Re-running often resolved it, so the underlying cause appears to be inconsistent artifact generation rather than an application bug. This project also revealed how easily uvicorn processes remain alive in the background, requiring manual cleanup to avoid "address already in use" errors. Despite these hurdles, the final system works reliably once ports and tool instructions are carefully managed.

### 1. Database Setup
We start by running database_setup.py included in the github repo to get the support.db, which will be the source database for this project.

In [1]:
!python database_setup.py

Connected to database: support.db
Tables created successfully!
Triggers created successfully!

DATABASE SCHEMA

CUSTOMERS TABLE:
------------------------------------------------------------
  id              INTEGER     
  name            TEXT       NOT NULL 
  email           TEXT        
  phone           TEXT        
  status          TEXT       NOT NULL DEFAULT 'active'
  created_at      TIMESTAMP   DEFAULT CURRENT_TIMESTAMP
  updated_at      TIMESTAMP   DEFAULT CURRENT_TIMESTAMP

TICKETS TABLE:
------------------------------------------------------------
  id              INTEGER     
  customer_id     INTEGER    NOT NULL 
  issue           TEXT       NOT NULL 
  status          TEXT       NOT NULL DEFAULT 'open'
  priority        TEXT       NOT NULL DEFAULT 'medium'
  created_at      DATETIME    DEFAULT CURRENT_TIMESTAMP

FOREIGN KEYS:
------------------------------------------------------------
  tickets.customer_id -> customers.id

Would you like to insert sample data? (y/n): y

#### 1.1 Inspect dataset

In [2]:
!ls

database_setup.py  sample_data	support.db


In [3]:
import sqlite3

conn = sqlite3.connect('support.db')
cursor = conn.cursor()

cursor.execute("SELECT * FROM CUSTOMERS LIMIT 1;")
rows = cursor.fetchall()
rows

[(1,
  'John Doe',
  'new@email.com',
  '+1-555-0101',
  'active',
  '2025-12-01 23:07:33',
  '2025-12-02 06:59:50')]

### 2. Build MCP Server
We will build flask server with SQLite connection and the following tool endpoints:
- get_customer(customer_id) - uses customers.id
- list_customers(status, limit) - uses customers.status
- update_customer(customer_id, data) - uses customers fields
- create_ticket(customer_id, issue, priority) - uses tickets fields
- get_customer_history(customer_id) - uses tickets.customer_id

In [4]:
# Install required packages
!pip install flask flask-cors requests termcolor pyngrok -q

print("All packages installed successfully!")
print("Installed: Flask (web server), Flask-CORS (cross-origin support), requests (HTTP client), termcolor (colored output), pyngrok (tunneling)")


All packages installed successfully!
Installed: Flask (web server), Flask-CORS (cross-origin support), requests (HTTP client), termcolor (colored output), pyngrok (tunneling)


In [5]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import sqlite3
import datetime

DB_PATH ='support.db'

app = Flask(__name__)
CORS(app)

# -----------------------------
# Utility: DB connection
# -----------------------------
def get_conn():
    return sqlite3.connect(DB_PATH)


# -----------------------------
# 1. get_customer(customer_id)
# -----------------------------
@app.route("/get_customer", methods=["POST"])
def get_customer():
    data = request.json
    customer_id = data.get("customer_id")

    conn = get_conn()
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM customers WHERE id=?", (customer_id,))
    row = cursor.fetchone()
    conn.close()

    if not row:
        return jsonify({"error": "Customer not found"}), 404

    customer = {
        "id": row[0],
        "name": row[1],
        "email": row[2],
        "phone": row[3],
        "status": row[4],
        "created_at": row[5],
        "updated_at": row[6]
    }
    return jsonify(customer)


# -----------------------------
# 2. list_customers(status, limit)
# -----------------------------
@app.route("/list_customers", methods=["POST"])
def list_customers():
    data = request.json
    status = data.get("status")
    limit = data.get("limit", 1000)

    conn = get_conn()
    cursor = conn.cursor()
    cursor.execute(
        "SELECT * FROM customers WHERE status=? LIMIT ?",
        (status, limit)
    )
    rows = cursor.fetchall()
    conn.close()

    customers = [
        {
            "id": r[0],
            "name": r[1],
            "email": r[2],
            "phone": r[3],
            "status": r[4],
            "created_at": r[5],
            "updated_at": r[6]
        }
        for r in rows
    ]

    return jsonify(customers)


# -----------------------------
# 3. update_customer(customer_id, data)
# -----------------------------
@app.route("/update_customer", methods=["POST"])
def update_customer():
    data = request.json
    customer_id = data.get("customer_id")
    update_data = data.get("data", {})

    fields = []
    vals = []

    for key, value in update_data.items():
        fields.append(f"{key}=?")
        vals.append(value)

    vals.append(customer_id)

    sql = f"UPDATE customers SET {', '.join(fields)}, updated_at=? WHERE id=?"
    vals.insert(-1, datetime.datetime.utcnow())

    conn = get_conn()
    cursor = conn.cursor()
    cursor.execute(sql, vals)
    conn.commit()
    conn.close()

    return jsonify({"status": "success"})


# -----------------------------
# 4. create_ticket(customer_id, issue, priority)
# -----------------------------
@app.route("/create_ticket", methods=["POST"])
def create_ticket():
    data = request.json
    customer_id = data.get("customer_id")
    issue = data.get("issue")
    priority = data.get("priority", "medium")

    conn = get_conn()
    cursor = conn.cursor()
    cursor.execute(
        """
        INSERT INTO tickets (customer_id, issue, status, priority, created_at)
        VALUES (?, ?, 'open', ?, ?)
        """,
        (customer_id, issue, priority, datetime.datetime.utcnow())
    )
    conn.commit()
    conn.close()

    return jsonify({"status": "ticket_created"})


# -----------------------------
# 5. get_customer_history(customer_id)
# -----------------------------
@app.route("/get_customer_history", methods=["POST"])
def get_customer_history():
    data = request.json
    customer_id = data.get("customer_id")

    conn = get_conn()
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM tickets WHERE customer_id=?", (customer_id,))
    rows = cursor.fetchall()
    conn.close()

    tickets = [
        {
            "id": r[0],
            "customer_id": r[1],
            "issue": r[2],
            "status": r[3],
            "priority": r[4],
            "created_at": r[5]
        }
        for r in rows
    ]

    return jsonify(tickets)


# -----------------------------
# Run the MCP server
# -----------------------------
@app.route("/")
def home():
    return "MCP server is running!", 200


#### 2.1 Runn server in the background
We need to set the server run in the background wirh Thread() due to limitation on Jupyter Notebook environment.

In [6]:
from threading import Thread

def run_server():
    app.run(port=5001)

Thread(target=run_server).start()

 * Serving Flask app '__main__'
 * Debug mode: off


#### 2.2 Localhost -> Public Link
We use ngrok here to configurate our local server to public link, so that we can test the link beyond the Jupyter Notebook backend environment.

In [7]:
!ngrok config add-authtoken 364uyk8s2zoAVPRhD17PtUoks91_7QGvH6ijDSpMCYWctPVZd

INFO:werkzeug:Press CTRL+C to quit


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [8]:
from pyngrok import ngrok

public_url = ngrok.connect(5001)
public_url

<NgrokTunnel: "https://unpathetic-antwan-malvasian.ngrok-free.dev" -> "http://localhost:5001">

#### 2.3 Test MCP
Since the request return what we expected, we can tell our MCP is well set on the air now.

In [9]:
import requests
public_url_only = "https://unpathetic-antwan-malvasian.ngrok-free.dev"
url = public_url_only + "/get_customer"
response = requests.post(url, json={"customer_id": 1})
response.json()

INFO:werkzeug:127.0.0.1 - - [02/Dec/2025 07:05:11] "POST /get_customer HTTP/1.1" 200 -


{'created_at': '2025-12-01 23:07:33',
 'email': 'new@email.com',
 'id': 1,
 'name': 'John Doe',
 'phone': '+1-555-0101',
 'status': 'active',
 'updated_at': '2025-12-02 06:59:50'}

### 3. Creat A2A coordinator framework
#### 3.1 Installation and Setup
Beyond pip install, we also need to setup new project in Google AI Lab for API key with the following link:

https://aistudio.google.com/app/api-keys

We can then set our public url and google api key as environment variable for easy access.

In [10]:
# Install required packages
%pip install --upgrade -q google-genai google-adk==1.9.0 a2a-sdk==0.3.0 python-dotenv aiohttp uvicorn requests mermaid-python nest-asyncio

In [11]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

##### 3.1.1 Environment Configuration

In [12]:
# Targeted workaround for google-adk==1.9.0 compatibility with a2a-sdk==0.3.0
# This cell shall be removed when google-adk releases the version next to >1.9.0
# (after https://github.com/google/adk-python/pull/2297)

import sys

from a2a.client import client as real_client_module
from a2a.client.card_resolver import A2ACardResolver

class PatchedClientModule:
    def __init__(self, real_module) -> None:
        for attr in dir(real_module):
            if not attr.startswith('_'):
                setattr(self, attr, getattr(real_module, attr))
        self.A2ACardResolver = A2ACardResolver


patched_module = PatchedClientModule(real_client_module)
sys.modules['a2a.client.client'] = patched_module  # type: ignore

In [13]:
import asyncio
import logging
import os
import sys
import threading
import time

from typing import Any

import httpx
import nest_asyncio
import uvicorn

from a2a.client import ClientConfig, ClientFactory, create_text_message_object
from a2a.server.apps import A2AStarletteApplication
from a2a.server.request_handlers import DefaultRequestHandler
from a2a.server.tasks import InMemoryTaskStore
from a2a.types import (
    AgentCapabilities,
    AgentCard,
    AgentSkill,
    TransportProtocol,
)
from a2a.utils.constants import AGENT_CARD_WELL_KNOWN_PATH
from dotenv import load_dotenv
from google.adk.a2a.executor.a2a_agent_executor import (
    A2aAgentExecutor,
    A2aAgentExecutorConfig,
)
from google.adk.agents import Agent, SequentialAgent
from google.adk.agents.remote_a2a_agent import RemoteA2aAgent
from google.adk.artifacts import InMemoryArtifactService
from google.adk.memory.in_memory_memory_service import InMemoryMemoryService
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search

/usr/local/lib/python3.12/dist-packages/google/cloud/aiplatform/models.py:52: FutureWarning: Support for google-cloud-storage < 3.0.0 will be removed in a future version of google-cloud-aiplatform. Please upgrade to google-cloud-storage >= 3.0.0.
  from google.cloud.aiplatform.utils import gcs_utils


In [14]:
# Set Google Cloud Configuration
os.environ['GOOGLE_GENAI_USE_VERTEXAI'] = 'FLASE'
os.environ['GOOGLE_CLOUD_PROJECT'] = (
    'gen-lang-client-0178360277'  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
)
os.environ['GOOGLE_CLOUD_LOCATION'] = (
    'us-central1'  # Replace with your location
)

load_dotenv()
from google.colab import userdata

os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')



print('Environment variables configured:')
print(f'GOOGLE_GENAI_USE_VERTEXAI: {os.environ["GOOGLE_GENAI_USE_VERTEXAI"]}')
print(f'GOOGLE_CLOUD_PROJECT: {os.environ["GOOGLE_CLOUD_PROJECT"]}')
print(f'GOOGLE_CLOUD_LOCATION: {os.environ["GOOGLE_CLOUD_LOCATION"]}')

Environment variables configured:
GOOGLE_GENAI_USE_VERTEXAI: FLASE
GOOGLE_CLOUD_PROJECT: gen-lang-client-0178360277
GOOGLE_CLOUD_LOCATION: us-central1


In [15]:
# Authenticate your notebook environment (Colab only)
if 'google.colab' in sys.modules:
    from google.colab import auth

    auth.authenticate_user(project_id=os.environ['GOOGLE_CLOUD_PROJECT'])

In [16]:
# Setup logging
logging.basicConfig(
    level=logging.ERROR,
    format='%(asctime)s - %(levelname)s - %(name)s - %(message)s',
)

#### 3.2 Creat Tools
We put previous funtionality we build with the database as function for our agents' tool box.

In [17]:
from typing import Any, Dict, Optional,List

# Environment variables
MCP_URL = 'https://unpathetic-antwan-malvasian.ngrok-free.dev'
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

# ---------------------------
# ADK Function Tools (Correct)
# ---------------------------

def get_customer(tool_context, customer_id: int):
    print("get_customer called with:", {"customer_id": customer_id})
    resp = requests.post(
        f"{MCP_URL}/get_customer",
        json={"customer_id": customer_id}
    )
    return resp.json()


def list_customers(tool_context, status: str, limit: int):
    print("list_customers called with:", {"status": status, "limit": limit})
    resp = requests.post(
        f"{MCP_URL}/list_customers",
        json={"status": status, "limit": limit}
    )
    return resp.json()


def update_customer(tool_context, customer_id: int, data: dict):
    print("update_customer called with:", {"customer_id": customer_id, "data": data})
    resp = requests.post(
        f"{MCP_URL}/update_customer",
        json={"customer_id": customer_id, "data": data}
    )
    return resp.json()


def create_ticket(tool_context, customer_id: int, issue: str, priority: str ):
    print("create_ticket called with:", {
        "customer_id": customer_id,
        "issue": issue,
        "priority": priority
    })
    resp = requests.post(
        f"{MCP_URL}/create_ticket",
        json={"customer_id": customer_id, "issue": issue, "priority": priority}
    )
    return resp.json()


def get_customer_history(tool_context, customer_id: int):
    print("get_customer_history called with:", {"customer_id": customer_id})
    resp = requests.post(
        f"{MCP_URL}/get_customer_history",
        json={"customer_id": customer_id}
    )
    return resp.json()


#### 3.3 Agent 1: Customer Data Agent

In [18]:
# Customer Data Agent
customer_data_agent = Agent(
    name="customer_data_agent",
    model="gemini-2.5-pro",
    #description="Handles customer data operations via MCP",
    instruction="""
    You are the Customer Data Agent.

    Your ONLY job is to retrieve or modify customer data USING THE MCP TOOLS PROVIDED.
    You MUST NEVER answer with just natural language unless a tool explicitly fails.

    Available database Schema and relevant tools:

    Table 1. Customers Table:

    id              INTEGER PRIMARY KEY
    name            TEXT NOT NULL
    email           TEXT
    phone           TEXT
    status          TEXT ('active' or 'disabled')
    created_at      TIMESTAMP
    updated_at      TIMESTAMP

    TOOLS AND THEIR REQUIRED PARAMETERS:

    1. get_customer(customer_id: int)
      - Use this when the user asks for a specific customer's information.
      - REQUIRED parameter: customer_id (integer)

    2. list_customers(status: str, limit: int)
      - Use this when the user asks for lists of customers filtered by status:
          "active", "disabled", or any status requested.
      - REQUIRED: status (string)
      - OPTIONAL: limit (integer), set huge number is fine

    3. update_customer(customer_id: int, data: dict)
      - Use this when the user wants to update information about a customer.
      - REQUIRED: customer_id (integer)
      - REQUIRED: data (dictionary of fields to update)

    Table 2. Tickets Table:
    id              INTEGER PRIMARY KEY
    customer_id     INTEGER (FK to customers.id)
    issue           TEXT NOT NULL
    status          TEXT ('open', 'in_progress', 'resolved')
    priority        TEXT ('low', 'medium', 'high')
    created_at      DATETIME

    TOOLS AND THEIR REQUIRED PARAMETERS:

    1. create_ticket(customer_id: int, issue: str, priority: str)
      - Use this if the user mentions an issue that requires a support ticket.
      - REQUIRED: customer_id (integer)
      - REQUIRED: issue (string describing the problem)
      - OPTIONAL: priority (string: "low", "medium", "high")

    2. get_customer_history(customer_id: int)
      - Use this when the user asks for a customer's ticket history from ticket table.
      - REQUIRED: customer_id (integer)

    RULES YOU MUST FOLLOW:
    - ALWAYS call a tool/ some tools. Never respond directly with text.
    - ALWAYS supply the correct parameters as shown above.
    - NEVER guess information.
    - If you cannot determine customer_id from context, ask the Router Agent for it.
    - If the user provides customer ID in text (e.g., “customer 123”), extract the integer.

    When the task need to join tables/ many information, call multiple tools if necessary.
    Your entire purpose is to take user intent → map to the correct tool → pass correct parameters.

      """,
    tools=[get_customer,
           list_customers,
           update_customer,
           create_ticket,
           get_customer_history],
)

In [19]:
customer_data_agent_card = AgentCard(
    name='Customer Data Agent',
    url='http://localhost:10020',
    description='Specialist agent that manages customer records and ticket history via MCP.',
    version='1.0',
    capabilities=AgentCapabilities(streaming=True),
    default_input_modes=['text/plain'],
    default_output_modes=['text/plain'],
    preferred_transport=TransportProtocol.jsonrpc,
    skills=[
        AgentSkill(
            id='customer_data_agent',
            name='Find Customer Data',
            description='Provides and manipulate customer record via mcp',
            tags=['data', 'customer id', 'customer history', 'status','issue','ticket'],
            examples=[
                'Retrieve ticket history',
                'Create tickets for customers',
                'Update customer records',
                'List customers by their status',
                'Look up individual customers by ID',
            ],
        )
    ],
)

In [20]:
remote_customer_data_agent = RemoteA2aAgent(
    name='customer_data_agent',
    description='Provides and manipulate customer record via mcp',
    agent_card=f'http://localhost:10020{AGENT_CARD_WELL_KNOWN_PATH}',
)

#### 3.3 Agent 2: Support Agent

In [21]:
# Create the Trend Analyzer ADK Agent
customer_support_agent = Agent(
    model='gemini-2.5-pro',
    name='customer_support_agent',
    instruction="""
    You are the Customer Support Agent in a customer service system.

    You:
    - Help with general customer support queries like account issues, upgrades, cancellations, and billing questions.
    - Escalate complex issues
    - customer information might be provided to you if available
    - Requests customer context or data update action from Data Agent when required
      - Data Agent could provide information about the following data:
        - Customer ID
        - Customer Name
        - Customer Email
        - Customer Phone
        - Customer Status
        - Customer Created At
        - Customer Updated At
        - Ticket ID
        - Ticket Issue
        - Ticket Status
        - Ticket Priority
        - Ticket Created At
    - Provides solutions and recommendations as customer service support

    Behavior:
    - If a customer ID is not found/ offered, clearly say so and ask for more info (email, name, etc.).
    - For urgent billing issues (charged twice, refund, immediately, urgent), create a high priority ticket.
    - Summarize what you did with the tools in a friendly, clear, helpful response.
    """,
    #tools=[google_search],
)


In [22]:
customer_support_agent_card = AgentCard(
    name='Customer Support Agent',
    url='http://localhost:10021',
    description='Handles general customer support, status update, billing, and escalation/ priority assessment.',
    version='1.0',
    capabilities=AgentCapabilities(streaming=True),
    default_input_modes=['text/plain'],
    default_output_modes=['text/plain'],
    preferred_transport=TransportProtocol.jsonrpc,
    skills=[
        AgentSkill(
            id='customer_support',
            name='Customer Support',
            description='Handles general customer support, status update, billing, and escalation/ priority assessment.',
            tags=['support', 'customer service', 'solution', 'recommendations','billing'],
            examples=[
                'Get high-priority tickets for these IDs',
                'Can you handle this cancellation and billing problem?',
                "I've been charged twice, please refund immediately!",
            ],
        )
    ],
)

In [23]:
remote_customer_support_agent = RemoteA2aAgent(
    name='customer_support',
    description='Handles general customer support, status update, billing, and escalation/ priority assessment.',
    agent_card=f'http://localhost:10021{AGENT_CARD_WELL_KNOWN_PATH}',
)

#### 3.3 Agent 3: Router Agent (Orchestrator)

In [24]:
host_agent = Agent(
    name="customer_host_agent",
    model="gemini-2.5-pro",
    instruction="""
You are the Router / Host Agent for a customer service system.

YOUR JOB:
- Break down complex queries into sub-tasks.
- Decide which sub-agent should handle each step.
- ALWAYS call the correct sub-agent via A2A.
- NEVER answer anything directly and dont make up process.

RULES:
- Do NOT guess data.
- Do NOT answer directly without A2A calls.
- ALWAYS call at least one A2A sub-agent per request.
- You must make multiple A2A calls when needed.
- If customer ID doesn't exist, say you dont find it or re-confirm with customer

Your output must always come from combining sub-agent tool results.
""",
    tools=[],  # host agent has no MCP tools
    sub_agents=[remote_customer_data_agent, remote_customer_support_agent]
)


In [25]:
host_agent_card = AgentCard(
    name='Customer Service Host',
    url='http://localhost:10022',
    description="Orchestrates customer data and support agents for full customer service flows.When query ask for information/question from database, you must present the relevant data",
    version='1.0',
    capabilities=AgentCapabilities(streaming=True),
    default_input_modes=['text/plain'],
    default_output_modes=['application/json'],
    preferred_transport=TransportProtocol.jsonrpc,
    skills=[
        AgentSkill(
            id='customer_service',
            name="Customer Service Orchestrator",
            description='Takes customer queries and uses the specialized agents to apply customer services; When query ask for information/question from database, you must present the relevant data',
            tags=['customer support', 'routing', 'orchestration', 'multi-agent'],
            examples=[
                "Get customer information for ID 5",
                "I'm customer 1 and need help upgrading my account",
                "Show me all active customers who have open tickets",
                "I've been charged twice, please refund immediately!",
            ],
        )
    ],
)

### 4. Running
We will start our agents and run the complete system.
#### 4.1 Starting the A2A Servers

In [26]:
def create_agent_a2a_server(agent, agent_card):
    """Create an A2A server for any ADK agent.

    Args:
        agent: The ADK agent instance
        agent_card: The ADK agent card

    Returns:
        A2AStarletteApplication instance
    """
    runner = Runner(
        app_name=agent.name,
        agent=agent,
        artifact_service=InMemoryArtifactService(),
        session_service=InMemorySessionService(),
        memory_service=InMemoryMemoryService(),
    )

    config = A2aAgentExecutorConfig()
    executor = A2aAgentExecutor(runner=runner, config=config)

    request_handler = DefaultRequestHandler(
        agent_executor=executor,
        task_store=InMemoryTaskStore(),
    )

    # Create A2A application
    return A2AStarletteApplication(
        agent_card=agent_card, http_handler=request_handler
    )

In [27]:
# Apply nest_asyncio
nest_asyncio.apply()

# Store server tasks
server_tasks: list[asyncio.Task] = []


async def run_agent_server(agent, agent_card, port) -> None:
    """Run a single agent server."""
    app = create_agent_a2a_server(agent, agent_card)

    config = uvicorn.Config(
        app.build(),
        host='127.0.0.1',
        port=port,
        log_level='warning',
        loop='none',  # Important: let uvicorn use the current loop
    )

    server = uvicorn.Server(config)
    await server.serve()


async def start_all_servers() -> None:
    """Start all servers in the same event loop."""
    # Create tasks for all servers
    tasks = [
        asyncio.create_task(
            run_agent_server(customer_data_agent, customer_data_agent_card, 10020)
        ),
        asyncio.create_task(
            run_agent_server(customer_support_agent, customer_support_agent_card, 10021)
        ),
        asyncio.create_task(
            run_agent_server(host_agent, host_agent_card, 10022)
        ),
    ]

    # Give servers time to start
    await asyncio.sleep(2)

    print('✅ All agent servers started!')
    print('   - Customer Data Agent: http://127.0.0.1:10020')
    print('   - Support Agent: http://127.0.0.1:10021')
    print('   - Host Agent: http://127.0.0.1:10022')

    # Keep servers running
    try:
        await asyncio.gather(*tasks)
    except KeyboardInterrupt:
        print('Shutting down servers...')


# Run in a background thread


def run_servers_in_background() -> None:
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    loop.run_until_complete(start_all_servers())


# Start the thread
server_thread = threading.Thread(target=run_servers_in_background, daemon=True)
server_thread.start()

# Wait for servers to be ready
time.sleep(3)

✅ All agent servers started!
   - Customer Data Agent: http://127.0.0.1:10020
   - Support Agent: http://127.0.0.1:10021
   - Host Agent: http://127.0.0.1:10022


In [28]:
print('Trending Agent Card:')
print(customer_data_agent_card)
print('\nAnalyzer Agent Card:')
print(customer_support_agent_card)
print('\nHost Agent Card:')
print(host_agent_card)

Trending Agent Card:
additional_interfaces=None capabilities=AgentCapabilities(extensions=None, push_notifications=None, state_transition_history=None, streaming=True) default_input_modes=['text/plain'] default_output_modes=['text/plain'] description='Specialist agent that manages customer records and ticket history via MCP.' documentation_url=None icon_url=None name='Customer Data Agent' preferred_transport='JSONRPC' protocol_version='0.3.0' provider=None security=None security_schemes=None signatures=None skills=[AgentSkill(description='Provides and manipulate customer record via mcp', examples=['Retrieve ticket history', 'Create tickets for customers', 'Update customer records', 'List customers by their status', 'Look up individual customers by ID'], id='customer_data_agent', input_modes=None, name='Find Customer Data', output_modes=None, security=None, tags=['data', 'customer id', 'customer history', 'status', 'issue', 'ticket'])] supports_authenticated_extended_card=None url='http

### 5. Testing the System
 Call the A2A agents (the 2 remote agents, and the host agent that refers to the 2 remote agents as sub agents)

In [29]:
class A2ASimpleClient:
    """A2A Simple to call A2A servers."""

    def __init__(self, default_timeout: float = 240.0):
        self._agent_info_cache: dict[
            str, dict[str, Any] | None
        ] = {}  # Cache for agent metadata
        self.default_timeout = default_timeout

    async def create_task(self, agent_url: str, message: str) -> str:
        """Send a message following the official A2A SDK pattern."""
        # Configure httpx client with timeout
        timeout_config = httpx.Timeout(
            timeout=self.default_timeout,
            connect=10.0,
            read=self.default_timeout,
            write=10.0,
            pool=5.0,
        )

        async with httpx.AsyncClient(timeout=timeout_config) as httpx_client:
            # Check if we have cached agent card data
            if (
                agent_url in self._agent_info_cache
                and self._agent_info_cache[agent_url] is not None
            ):
                agent_card_data = self._agent_info_cache[agent_url]
            else:
                # Fetch the agent card
                agent_card_response = await httpx_client.get(
                    f'{agent_url}{AGENT_CARD_WELL_KNOWN_PATH}'
                )
                agent_card_data = self._agent_info_cache[agent_url] = (
                    agent_card_response.json()
                )

            # Create AgentCard from data
            agent_card = AgentCard(**agent_card_data)

            # Create A2A client with the agent card
            config = ClientConfig(
                httpx_client=httpx_client,
                supported_transports=[
                    TransportProtocol.jsonrpc,
                    TransportProtocol.http_json,
                ],
                use_client_preference=True,
            )

            factory = ClientFactory(config)
            client = factory.create(agent_card)

            # Create the message object
            message_obj = create_text_message_object(content=message)

            # Send the message and collect responses
            responses = []
            async for response in client.send_message(message_obj):
                responses.append(response)

            # The response is a tuple - get the first element (Task object)
            if (
                responses
                and isinstance(responses[0], tuple)
                and len(responses[0]) > 0
            ):
                task = responses[0][0]  # First element of the tuple

                # Extract text: task.artifacts[0].parts[0].root.text
                try:
                    return task.artifacts[0].parts[0].root.text
                except (AttributeError, IndexError):
                    return str(task)

            return 'No response received'

In [30]:
a2a_client = A2ASimpleClient()

In [34]:
# Customer Data Retrive
async def test_customer_service() -> None:
    """Test customer service agent."""
    customer_service = await a2a_client.create_task(
        'http://localhost:10020', "Get customer information for ID 1"
    )
    print(customer_service)


# Run the async function
asyncio.run(test_customer_service())

get_customer called with: {'customer_id': 1}


INFO:werkzeug:127.0.0.1 - - [02/Dec/2025 07:08:19] "POST /get_customer HTTP/1.1" 200 -


I have retrieved the customer information for ID 1.

**Customer Details:**
*   **ID:** 1
*   **Name:** John Doe
*   **Email:** new@email.com
*   **Phone:** +1-555-0101
*   **Status:** active
*   **Created At:** 2025-12-01 23:07:33
*   **Updated At:** 2025-12-02 06:59:50


In [35]:
# Customer Support
async def test_customer_service() -> None:
    """Test customer service agent."""
    customer_service = await a2a_client.create_task(
        'http://localhost:10021', "i want to refund my product"
    )
    print(customer_service)


# Run the async function
asyncio.run(test_customer_service())

I can certainly help you with that. To get started, could you please provide me with your email address or customer ID associated with the purchase?


In [36]:
# Host Agent
async def test_customer_service() -> None:
    """Test customer service agent."""
    customer_service = await a2a_client.create_task(
        'http://localhost:10022', "can i get refund for my recent purchase?"
    )
    print(customer_service)


# Run the async function
asyncio.run(test_customer_service())

Of course, I can help you with your refund request.

Since this is a billing issue, I'll create a high-priority ticket for our billing team to review and process it for you.

Could you please provide me with your email address or customer ID so I can locate your account and recent purchase?


#### 5.0 In Case we need to update Agent
we will need to kill old servers to make sure the new version of tools/ agents are loaded.

In [ ]:
#!pkill -f uvicorn

In [ ]:
#!pkill -f "127.0.0.1:100"

In [ ]:
# reset runtime state
#import gc, asyncio
#gc.collect()
#asyncio.get_event_loop().stop()

### 6. Testing Scenario


#### 6.1 Simple Query: "Get customer information for ID 5"
Single agent, straightforward MCP call

In [37]:
# Host Agent
async def test_customer_service() -> None:
    """Test customer service agent."""
    customer_service = await a2a_client.create_task(
        'http://localhost:10022', "Get customer information for ID 1"
    )
    print(customer_service)


# Run the async function
asyncio.run(test_customer_service())

get_customer called with: {'customer_id': 1}


INFO:werkzeug:127.0.0.1 - - [02/Dec/2025 07:09:12] "POST /get_customer HTTP/1.1" 200 -


I have retrieved the customer information for ID 1.

**Customer Details:**
*   **ID:** 1
*   **Name:** John Doe
*   **Email:** new@email.com
*   **Phone:** +1-555-0101
*   **Status:** active
*   **Created At:** 2025-12-01 23:07:33
*   **Updated At:** 2025-12-02 06:59:50



#### 6.2 Coordinated Query: "I'm customer 12345 and need help upgrading my account"
Multiple agents coordinate: data fetch + support response

In [43]:
# Host Agent
async def test_customer_service() -> None:
    """Test customer service agent."""
    customer_service = await a2a_client.create_task(
        'http://localhost:10022',
        "I'm customer 12345 and need help upgrading my account"
    )
    print(customer_service)


# Run the async function
asyncio.run(test_customer_service())

create_ticket called with: {'customer_id': 12345, 'issue': 'needs help upgrading account', 'priority': 'medium'}


/tmp/ipython-input-448448840.py:128: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  (customer_id, issue, priority, datetime.datetime.utcnow())
/tmp/ipython-input-448448840.py:123: DeprecationWarning: The default datetime adapter is deprecated as of Python 3.12; see the sqlite3 documentation for suggested replacement recipes
  cursor.execute(
INFO:werkzeug:127.0.0.1 - - [02/Dec/2025 07:13:19] "POST /create_ticket HTTP/1.1" 200 -


An error occurred during processing


#### 6.3 Complex Query: "Show me all active customers who have open tickets"
Requires negotiation between data and support agents

In [45]:
# Host Agent
async def test_customer_service() -> None:
    """Test customer service agent."""
    customer_service = await a2a_client.create_task(
        'http://localhost:10022',
        "Show me all active customers who have open tickets"
    )
    print(customer_service)


# Run the async function
asyncio.run(test_customer_service())

list_customers called with: {'status': 'active', 'limit': 1000}


INFO:werkzeug:127.0.0.1 - - [02/Dec/2025 07:17:00] "POST /list_customers HTTP/1.1" 200 -


get_customer_history called with: {'customer_id': 1}


INFO:werkzeug:127.0.0.1 - - [02/Dec/2025 07:17:14] "POST /get_customer_history HTTP/1.1" 200 -


get_customer_history called with: {'customer_id': 2}


...

get_customer_history called with: {'customer_id': 104}


INFO:werkzeug:127.0.0.1 - - [02/Dec/2025 07:17:40] "POST /get_customer_history HTTP/1.1" 200 -


get_customer_history called with: {'customer_id': 105}


INFO:werkzeug:127.0.0.1 - - [02/Dec/2025 07:17:41] "POST /get_customer_history HTTP/1.1" 200 -


| id | name | email | phone |
| --- | --- | --- | --- |
| 1 | John Doe | new@email.com | +1-555-0101 |
| 2 | Jane Smith | jane.smith@example.com | +1-555-0102 |
| 4 | Alice Williams | alice.w@techcorp.com | +1-555-0104 |
| 5 | Charlie Brown | charlie.brown@email.com | +1-555-0105 |
| 6 | Diana Prince | diana.prince@company.org | +1-555-0106 |
| 7 | Edward Norton | e.norton@business.net | +1-555-0107 |
| 9 | George Miller | george.m@enterprise.com | +1-555-0109 |
| 10 | Hannah Lee | hannah.lee@global.com | +1-555-0110 |
| 11 | Isaac Newton | isaac.n@science.edu | +1-555-0111 |
| 12 | Julia Roberts | julia.r@movies.com | +1-555-0112 |
| 15 | Michael Scott | michael.scott@paper.com | +1-555-0115 |


#### 6.4 Escalation: "I've been charged twice, please refund immediately!"
Router must identify urgency and route appropriately

In [41]:
# Host Agent
async def test_customer_service() -> None:
    """Test customer service agent."""
    customer_service = await a2a_client.create_task(
        'http://localhost:10022', "I've been charged twice, please refund immediately!"
    )
    print(customer_service)


# Run the async function
asyncio.run(test_customer_service())

I'm very sorry to hear that you've been charged twice. I understand this is urgent, and I will treat this with the highest priority.

To proceed with the refund, I need to locate your account. Could you please provide me with your email address or customer ID?

Once I have your information, I will create a high-priority ticket and escalate it to our billing team for an immediate refund.


#### 6.5 Multi-Intent: "Update my email to new@email.com and show my ticket history"
Parallel task execution and coordination

In [42]:
# Host Agent
async def test_customer_service() -> None:
    """Test customer service agent."""
    customer_service = await a2a_client.create_task(
        'http://localhost:10022', "Update my email to new@email.com and show my ticket history"
    )
    print(customer_service)


# Run the async function
asyncio.run(test_customer_service())

Of course, I can help with that.

Before I can update your email and pull up your ticket history, I need to find your account. Could you please provide your Customer ID or the email address currently on file?
